In [ ]:
from mainv3 import SystemDesign

In [ ]:
from vae_model import VAEWithFeatureSelection

# Create and setup the system with VAE model
system = SystemDesign(
    system_columns=['solvent_1', 'solvent_2', 'temperature'],
    raw_data_path='curve_fit_results_x_is_7.csv',
    extra_fitted_points=1,
    target_columns=['J0', 'J1', 'J2']
)

# Train the VAE model
system.train_model(
    model_class=VAEWithFeatureSelection,
    feature_selection_method='random_forest',
    n_features=10,
    keep_prefixes=['solvent_1_pure', 'solvent_2_pure', 'system', 'solubility_', 'temperature'],
    epochs=50,
    batch_size=32,
    verbose=1,
    optimize_hyperparams=False,
    n_calls=11,
    latent_dim=16,
    kl_weight=0.001
)

In [ ]:
system.save('vae_model.pkl')

In [ ]:
system = system.load('vae_model.pkl')

In [ ]:
from neural_network_model import NeuralNetworkWithFeatureSelection

# Create and setup the system
system = SystemDesign(
    system_columns=['solvent_1','solvent_2','temperature'],
    raw_data_path='curve_fit_results_x_is_7.csv',
    extra_fitted_points=1,
    target_columns=['J0','J1','J2']
)

# Train the model
system.train_model(
    model_class=NeuralNetworkWithFeatureSelection,
    feature_selection_method='bayesian',
    n_features=1000,
    keep_prefixes=['solvent_1_pure','solvent_2_pure','system','solubility_','temperature'],
    epochs=1000, 
    batch_size=32, 
    verbose=1,
    optimize_hyperparams=True,
    n_calls=11
)

# Evaluate the model
system.evaluate_model()

# Get predictions and metrics
predictions, actuals, mae = system.get_predictions_and_metrics()

In [ ]:
system.save("NN_model.pkl")

In [ ]:
system.load("NN_model.pkl")

In [ ]:
from xgb_model import XGBoostWithFeatureSelection

# Create and setup the system with XGBoost model
system = SystemDesign(
    system_columns=['solvent_1', 'solvent_2', 'temperature'],
    raw_data_path='curve_fit_results_x_is_7.csv',
    extra_fitted_points=1,
    target_columns=['J0', 'J1', 'J2']
)

# Train the XGBoost model
system.train_model(
    model_class=XGBoostWithFeatureSelection,
    feature_selection_method='random_forest',
    n_features=10,
    keep_prefixes=['solvent_1_pure', 'solvent_2_pure', 'system', 'solubility_', 'temperature'],
    verbose=1,
    optimize_hyperparams=False,
    n_calls=11,
    n_estimators=100,
    max_depth=6
)

# Evaluate the model
system.evaluate_model()

# Get predictions and metrics
predictions, actuals, mae = system.get_predictions_and_metrics()

# Plot feature importance
system.model.plot_feature_importance(top_n=15)


In [ ]:
system.save("XGB_model.pkl")

In [ ]:
system.load("XGB_model.pkl")